In [1]:
from symengine import *
import sympy
import numpy as np
import re
from joblib import Parallel, delayed
from joblib import parallel_backend
from joblib import wrap_non_picklable_objects
import sys
print(sys.executable)

/store/SOFT/gt369/Software/anaconda3/envs/pystokes/bin/python


Number of periodic boxes and Fourier modes

### Definitions and auxiliary functions

In [33]:
xij, yij, zij = symbols("xij, yij, zij", real=True) 
b, eta = symbols("b, eta", real=True, positive=True)

# indices for summation:
## Caution, cannot use lambda, instead use Lambda
## do not use eta as index, it's the viscosity
alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma = symbols(
    'alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma', integer=True)

#new symbols for periodic calculation: boxSize, xi (convergence parameter)
L, xi = symbols("L, xi", real=True, positive=True)
kx, ky, kz = symbols("kx, ky, kz", real=True) 

r0 = symbols("r0", real=True, positive=True) ## for limit r->0 in M2

rij = Matrix([xij, yij, zij])
k = Matrix([kx, ky, kz])
kDOTrij = xij*kx + yij*ky + zij*kz
norm_rij = sqrt(xij**2 + yij**2 + zij**2)
norm_k = sqrt(kx**2 + ky**2 + kz**2)

k0 = 2*pi/L

def norm(r):
    result=0
    for i in range(3):
        result += r[i]**2
    return sqrt(result)

KroneckerDelta = eye(3)

def eps(alpha, beta, gamma):
    return LeviCivita(alpha,beta,gamma)

### Calculus auxiliary functions

In [3]:
def Laplacian(G):
    return diff(G, rij[0], rij[0]) + diff(G, rij[1], rij[1]) + diff(G, rij[2], rij[2])

def Curl(G, alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(alpha,i,j)*diff(G(j,beta),rij[i])
    return result

def tilde_Curl(G, alpha, beta): ##introduce for convenience, minus sign already taken into account in matrix elements
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(beta,i,j)*diff(G(alpha, j),rij[i])
    return result

def CurlCurl(G, alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    result += eps(alpha,i,j)*eps(beta,k,l)*diff(G(j,l), rij[i], rij[k])
    return result


def SumG(G, sigma, eta2, mu):
    result=0
    for i in range(3):
        result += eps(sigma, eta2, i)*Laplacian( G(mu, i) )
    return result


def SumG3s1Sym(G, mu, nu2, alpha, gamma1, gamma2):
    result=0
    for i in range(3):
        result += eps(mu,nu2,i)*g3s1Sym(G, alpha,gamma1,gamma2,i) ##add argument G
    return result

### M2(r=0) function

In [4]:
def func_M2(alpha, beta):
    func = norm_rij*erf(xi*norm_rij)
    return KroneckerDelta[alpha, beta]*Laplacian(func) - diff( func, rij[alpha], rij[beta])

In [5]:
matM2 = zeros(3,3)
for i in range(3):
    for j in range(3):
        matM2[i,j] = func_M2(i,j)

def M2_(alpha,beta):
    return matM2[alpha,beta]

In [6]:
##auxiliary functions
def delG(G, alpha, beta, gamma):
    return diff( G(alpha,beta), rij[gamma])

def lapG(G, alpha,beta):
    return Laplacian( G(alpha,beta) )

def dellapG(G, alpha, beta, kappa1):
    return diff(lapG(G, alpha, beta), rij[kappa1])

def deldelG(G, zeta, beta, kappa1, nu):
    return diff(delG(G, zeta, beta, kappa1), rij[nu])

def deldellapG(G, alpha, beta, gamma1, kappa1):
    return diff(dellapG(G, alpha,beta, gamma1), rij[kappa1])

def deldeldelG(G, alpha, beta, gamma1, gamma2, gamma3):
    return diff(deldelG(G, alpha, beta, gamma1, gamma2), rij[gamma3])

def deldeldeldelG(G, alpha, beta, gamma1, gamma2, gamma3, gamma4):
    return diff(deldeldelG(G, alpha, beta, gamma1, gamma2, gamma3), rij[gamma4])

In [7]:
# GLL  ##simplify later, when filling matrix elements, after subsituting for xij -- much faster
def G1s1s(G, alpha, beta):
    return G(alpha, beta) + b**2/3*lapG(G, alpha, beta)

def G1s2a(G, alpha, beta):
    return -b/2*tilde_Curl(G, alpha,beta)

def G2a1s(G, alpha, beta):
    return b*Curl(G, alpha,beta)

def G2a2a(G, alpha, beta):
    return -b**2/2*CurlCurl(G, alpha,beta)

In [8]:
## GL2s and G2sL
def G1s2s(G, alpha, kappa1, beta):
    return -b/2*((delG(G, alpha, beta, kappa1) + delG(G, alpha, kappa1, beta)) + 4*b**2/15*(dellapG(G, alpha, beta, kappa1) + dellapG(G, alpha, kappa1, beta)))

def G2a2s(G, alpha, kappa1, beta):
    return -b**2/2*(diff(Curl(G, alpha, beta),rij[kappa1]) + diff(Curl(G, alpha, kappa1),rij[beta]))

def G2s1s(G, alpha, gamma1, beta):
    return b/2*((delG(G, alpha, beta, gamma1) + delG(G, gamma1, beta, alpha)) + 4*b**2/15*(dellapG(G, alpha,beta,gamma1) + dellapG(G, gamma1, beta, alpha)))

def G2s2a(G, alpha, gamma1, mu):
    return -b**2/4*(diff(tilde_Curl(G, gamma1,mu), rij[alpha]) + diff(tilde_Curl(G, alpha,mu), rij[gamma1])) #sign change from -> eps(mu, kappa1, beta)


## G2s2s
def G2s2s(G, alpha, gamma1, kappa1, beta):
    return -b**2/4*((deldelG(G, alpha, beta, gamma1, kappa1) + deldelG(G, gamma1, beta, alpha, kappa1)) + 
                      (deldelG(G, alpha,kappa1, gamma1, beta) + deldelG(G, gamma1, kappa1, alpha, beta)) +
                      b**2/5*(deldellapG(G, alpha,beta,gamma1,kappa1) + deldellapG(G, gamma1,beta,alpha,kappa1)) + 
                      b**2/5*(deldellapG(G, alpha, kappa1, gamma1, beta) + deldellapG(G, gamma1,kappa1,alpha,beta)))


## higher matrix elements
def G1s3t(G, alpha, beta):
    return -b**2/5*lapG(G, alpha,beta)


# symmetry for matrix elements of form l sigma, 3s
def g13sSym(G, alpha,kappa2,kappa1,beta):
    return (deldelG(G, alpha,beta,kappa1,kappa2) + deldelG(G, alpha,kappa2,kappa1,beta) + deldelG(G, alpha,kappa1,beta,kappa2))/3 - (
        KroneckerDelta[kappa1,kappa2]*lapG(G, alpha,beta) + KroneckerDelta[kappa1,beta]*lapG(G, alpha,kappa2) + KroneckerDelta[beta,kappa2]*lapG(G, alpha,kappa1))/15

def G1s3s(G, alpha,kappa2, kappa1, beta):
    return b**2*(g13sSym(G, alpha,kappa2,kappa1,beta) + 5*b**2/21*Laplacian(g13sSym(G, alpha,kappa2,kappa1,beta)))


# symmetry of matrix elements of form 3s, l sigma
def g3s1Sym(G, alpha,gamma1,gamma2,beta):
    return (deldelG(G, alpha,beta,gamma1,gamma2) + deldelG(G, gamma1,beta,gamma2,alpha) + deldelG(G, gamma2,beta,alpha,gamma1))/3 - (
        KroneckerDelta[gamma1, gamma2]*lapG(G, alpha,beta) + KroneckerDelta[gamma2,alpha]*lapG(G, gamma1,beta) + KroneckerDelta[alpha,gamma1]*lapG(G, gamma2,beta))/15

def G3s1s(G, alpha,gamma1,gamma2,beta):
    return b**2*(g3s1Sym(G, alpha,gamma1,gamma2,beta) + 5*b**2/21*Laplacian(g3s1Sym(G, alpha,gamma1,gamma2,beta)))

def G3a1s(G, Lambda, kappa2, beta):
    return b**2/2*(diff(Curl(G, Lambda, beta), rij[kappa2]) + diff(Curl(G, kappa2,beta), rij[Lambda]))

def G1s3a(G, alpha, nu2, sigma):
    return 2*b**2/3*(diff(tilde_Curl(G, alpha,nu2),rij[sigma]) - SumG(G, nu2,sigma,alpha)/3) # sign change due to --> eps(sigma,nu1,beta)


def g3sCurlSym(G, alpha,gamma1,gamma2,mu):
    return (diff(tilde_Curl(G, alpha,mu), rij[gamma1],rij[gamma2]) + diff(tilde_Curl(G, gamma1,mu), rij[gamma2],rij[alpha]) + diff(tilde_Curl(G, gamma2,mu), rij[alpha],rij[gamma1]))/3
        
def G3s2a(G, alpha,gamma1,gamma2,mu):
    return -b**3/2*g3sCurlSym(G, alpha, gamma1, gamma2, mu)## sign change due to -> eps(mu, kappa1, beta)

def G3a2a(G, Lambda, kappa2, mu):
    return -b**3/4*(diff(CurlCurl(G, Lambda,mu), rij[kappa2]) + diff(CurlCurl(G, kappa2,mu), rij[Lambda])) #sign change -> eps(mu,kappa1,beta)

def G2s3t(G, alpha,gamma1,nu2):
    return -b**3/10*(dellapG(G, alpha,nu2,gamma1) + dellapG(G, gamma1,nu2,alpha))

def G3s3t(G, alpha,gamma1,gamma2,nu2):
    return -b**4/5*Laplacian(g3s1Sym(G, alpha,gamma1,gamma2,nu2))

def G2s3a(G, alpha,gamma1,mu,nu2): #sign change -> eps(mu,nu1,beta)
    return b**3/3*( diff(tilde_Curl(G, alpha,mu), rij[gamma1], rij[nu2]) - diff(SumG(G, mu,nu2,alpha), rij[gamma1])/3
                   + diff(tilde_Curl(G, gamma1,mu), rij[alpha], rij[nu2]) - diff(SumG(G, mu,nu2,gamma1), rij[alpha])/3)

def G3a3a(G, Lambda, kappa2, mu, nu2): #sign change -> eps(mu,nu1,beta)
    return b**4/3*(diff(CurlCurl(G, Lambda,mu), rij[nu2], rij[kappa2]) + diff(CurlCurl(G, kappa2,mu), rij[nu2], rij[Lambda]))

def G3s3a(G, alpha,gamma1,gamma2,mu,nu2): #sign change -> eps(mu,nu1,beta)
    return 2*b**4/3*(diff(g3sCurlSym(G, alpha,gamma1,gamma2,mu), rij[nu2]) - Laplacian(SumG3s1Sym(G, mu,nu2,alpha,gamma1,gamma2))/3)


def g2s3sSym(G, alpha,gamma1,kappa2, kappa1, beta):
    return diff(g13sSym(G, alpha,kappa2,kappa1,beta), rij[gamma1]) + diff(g13sSym(G, gamma1, kappa2, kappa1, beta), rij[alpha])

def G2s3s(G, alpha,gamma1,kappa2, kappa1, beta):
    return b**3/2*(g2s3sSym(G, alpha,gamma1,kappa2, kappa1, beta) + 6*b**2/35*Laplacian(g2s3sSym(G, alpha,gamma1,kappa2, kappa1, beta)))


def gCurl3sSym(G, tau, kappa2, kappa1, beta):
    return (diff(Curl(G, tau, beta), rij[kappa1],rij[kappa2]) + diff(Curl(G, tau,kappa1), rij[kappa2],rij[beta]) + diff(Curl(G, tau,kappa2), rij[beta],rij[kappa1]))/3 

def G3a3s(G, Lambda, eta2, kappa2, kappa1, beta):
    return b**4/2*(diff(gCurl3sSym(G, Lambda, kappa1, kappa2, beta), rij[eta2]) + diff(gCurl3sSym(G, eta2, kappa1, kappa2, beta), rij[Lambda]))


def deldelg13sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return diff(g13sSym(G, alpha,kappa2,kappa1,beta), rij[gamma1], rij[gamma2])

def lapg13sSym(G, alpha,kappa2,kappa1,beta):
    return Laplacian(g13sSym(G, alpha,kappa2,kappa1,beta))

def g3s3sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return (deldelg13sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta) + deldelg13sSym(G, gamma1,gamma2,alpha,kappa2,kappa1,beta) + 
                deldelg13sSym(G, gamma2,alpha,gamma1,kappa2,kappa1,beta))/3 - (
                KroneckerDelta[gamma1,gamma2]*lapg13sSym(G, alpha, kappa2, kappa1, beta) + 
                KroneckerDelta[gamma2,alpha]*lapg13sSym(G, gamma1, kappa2, kappa1, beta) + 
                KroneckerDelta[alpha,gamma1]*lapg13sSym(G, gamma2, kappa2, kappa1, beta))/15

def G3s3s(G, alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return b**4*(g3s3sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta) + b**2/7*Laplacian(g3s3sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta)))

def G3a2s(G, Lambda, kappa2, kappa1, beta):
    return -b**3/4*(diff(Curl(G, Lambda,beta), rij[kappa1], rij[kappa2]) + diff(Curl(G, Lambda,kappa1), rij[beta], rij[kappa2])
                       + diff(Curl(G, kappa2,beta), rij[kappa1], rij[Lambda]) + diff(Curl(G, kappa2,kappa1), rij[beta], rij[Lambda]))


def g3s2sSym(G, alpha,gamma1,gamma2, kappa1, beta):
    return diff(g3s1Sym(G, alpha,gamma1,gamma2,beta), rij[kappa1]) + diff(g3s1Sym(G, alpha,gamma1,gamma2,kappa1), rij[beta])

def G3s2s(G, alpha,gamma1,gamma2,kappa1,beta):
    return -b**3/2*(g3s2sSym(G, alpha,gamma1,gamma2, kappa1, beta) + 6*b**2/35*Laplacian(g3s2sSym(G, alpha,gamma1,gamma2, kappa1, beta)))

def G2a3a(G, Lambda, mu, nu2): # sign change -> eps(mu,nu1,beta)
    return 2*b**3/3*diff(CurlCurl(G, Lambda, mu), rij[nu2])

def G2a3s(G, Lambda,kappa2,kappa1,beta):
    return b**3*gCurl3sSym(G, Lambda, kappa1, kappa2, beta)

In [9]:
## zero elements
def G3t1s(G, alpha,beta):
    return S(0)

def G3t2a(G, alpha,beta):
    return S(0)

def G3t2s(G, alpha, beta, kappa1):
    return S(0)

def G3t3t(G, alpha,beta):
    return S(0)

def G3t3a(G, alpha, beta, kappa1):
    return S(0)

def G3t3s(G, alpha, beta, kappa1, kappa2):
    return S(0)

def G2a3t(G, alpha,beta):
    return S(0)

def G3a3t(G, alpha, gamma1, beta):
    return S(0)

## Matrix assembly

In [10]:
@delayed
@wrap_non_picklable_objects
def matG(G, *args):
    return G(*args)

In [11]:
n_jobs = 50 

In [12]:
modes = ['1s', '2a', '2s', '3t', '3a', '3s']
modesV = ['2s', '3t'] ##possible slip modes, for double-layer matrix elements
SLDL  = ['G']#, 'K']
dim   = {'1s':3, '2a':3, '2s':5, '3t':3, '3a':5, '3s':7}

### 1) Fourier part of Green's function

In [13]:
%%time
sldl = 'G' # single-layer
GRF = 'M2_' # Fourier space part of Green's function
with parallel_backend('threading'): ## threading is quicker here: less overhead
    for mode1 in modes:
        for mode2 in modes:
            if mode1 in ['1s', '2a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj) for ki in range(3) 
                                                                                                        for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:                ## GLH needs symmetry factors
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj, lj) for ki in range(3)
                                                                                                            for kj in range(2)
                                                                                                                for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj, lj, mj) for ki in range(3)
                                                                                                                for kj in range(2)
                                                                                                                    for lj in range(kj,2)
                                                                                                                        for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')



            elif mode1 == '3t':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj) for ki in range(3) 
                                                                                                        for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj, lj) for ki in range(3)
                                                                                                            for kj in range(2)
                                                                                                                for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, kj, lj, mj) for ki in range(3)
                                                                                                                for kj in range(2)
                                                                                                                    for lj in range(kj,2)
                                                                                                                        for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')



            elif mode1 in ['2s', '3a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, kj) for ki in range(2) 
                                                                                                            for li in range(ki,3)
                                                                                                                for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, kj, lj) for ki in range(2)
                                                                                                                for li in range(ki,3)
                                                                                                                    for kj in range(2)
                                                                                                                        for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, kj, lj, mj) for ki in range(2)
                                                                                                                    for li in range(ki,3)
                                                                                                                        for kj in range(2)
                                                                                                                            for lj in range(kj,2)
                                                                                                                                for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')



            elif mode1 == '3s':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, mi, kj) for ki in range(2) 
                                                                                                                for li in range(ki,2)
                                                                                                                    for mi in range(li,3)
                                                                                                                        for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, mi, kj, lj) for ki in range(2)
                                                                                                                    for li in range(ki,2)
                                                                                                                        for mi in range(li,3)
                                                                                                                            for kj in range(2)
                                                                                                                                for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        matG(globals()[sldl+mode1+mode2], globals()[GRF], ki, li, mi, kj, lj, mj) for ki in range(2)
                                                                                                                        for li in range(ki,2)
                                                                                                                            for mi in range(li,3)
                                                                                                                                for kj in range(2)
                                                                                                                                    for lj in range(kj,2)
                                                                                                                                        for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

M2_1s1s done
M2_1s2a done
M2_1s2s done
M2_1s3t done
M2_1s3a done
M2_1s3s done
M2_2a1s done
M2_2a2a done
M2_2a2s done
M2_2a3t done
M2_2a3a done
M2_2a3s done
M2_2s1s done
M2_2s2a done
M2_2s2s done
M2_2s3t done
M2_2s3a done
M2_2s3s done
M2_3t1s done
M2_3t2a done
M2_3t2s done
M2_3t3t done
M2_3t3a done
M2_3t3s done
M2_3a1s done
M2_3a2a done
M2_3a2s done
M2_3a3t done
M2_3a3a done
M2_3a3s done
M2_3s1s done
M2_3s2a done
M2_3s2s done
M2_3s3t done
M2_3s3a done
M2_3s3s done
CPU times: user 10.7 s, sys: 591 ms, total: 11.3 s
Wall time: 11.4 s


In [14]:
## do not pass matrix as argument, this will not be picklable. String is okay. 
def limitElementM(Mstring, alpha, beta): ## extract an element from the matrix G for parallelisation
    return sympy.limit(sympy.powdenest((globals()['mat'+Mstring][alpha,beta]), force=True).subs({xij:r0, yij:r0, zij:r0}), r0, 0)

In [15]:
limitElementM('M2_1s1s', 0,0)

(-160*b**2*xi**3/9 + 8*xi)/sqrt(pi)

In [16]:
%%time
sldl = 'G' # single-layer
GRF = 'M2_' # Fourier space part of Green's function
with parallel_backend('multiprocessing'): ## multiprocessing is necessary here for simplification of each element on a different core
    for mode1 in modes:
        for mode2 in modes:
            if mode1 in ['1s', '2a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(limitElementM)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:                ## GLH needs symmetry factors
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(limitElementM)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(limitElementM)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')



            elif mode1 == '3t':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(limitElementM)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(limitElementM)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(limitElementM)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')



            elif mode1 in ['2s', '3a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(limitElementM)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(limitElementM)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(limitElementM)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')



            elif mode1 == '3s':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(limitElementM)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(limitElementM)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['listSimp'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(limitElementM)(GRF+mode1+mode2, alpha, beta) for alpha in range(dim[mode1]) for beta in range(dim[mode2]))
                    globals()['simplified'+GRF+mode1+mode2] = Matrix(globals()['listSimp'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(GRF+mode1+mode2+' done')

M2_1s1s done
M2_1s2a done
M2_1s2s done
M2_1s3t done
M2_1s3a done
M2_1s3s done
M2_2a1s done
M2_2a2a done
M2_2a2s done
M2_2a3t done
M2_2a3a done
M2_2a3s done
M2_2s1s done
M2_2s2a done
M2_2s2s done
M2_2s3t done
M2_2s3a done
M2_2s3s done
M2_3t1s done
M2_3t2a done
M2_3t2s done
M2_3t3t done
M2_3t3a done
M2_3t3s done
M2_3a1s done
M2_3a2a done
M2_3a2s done
M2_3a3t done
M2_3a3a done
M2_3a3s done
M2_3s1s done
M2_3s2a done
M2_3s2s done
M2_3s3t done
M2_3s3a done
M2_3s3s done
CPU times: user 6.43 s, sys: 13.1 s, total: 19.6 s
Wall time: 2min 42s


### 3) add Green's functions

In [19]:
GRF = "M2_"
for mode1 in modes:
    for mode2 in modes:
        globals()['matG'+GRF+mode1+mode2] = 1/(8*pi*eta) * globals()['simplified'+GRF+mode1+mode2]
        print(sldl+GRF+mode1+mode2+' done')

GM2_1s1s done
GM2_1s2a done
GM2_1s2s done
GM2_1s3t done
GM2_1s3a done
GM2_1s3s done
GM2_2a1s done
GM2_2a2a done
GM2_2a2s done
GM2_2a3t done
GM2_2a3a done
GM2_2a3s done
GM2_2s1s done
GM2_2s2a done
GM2_2s2s done
GM2_2s3t done
GM2_2s3a done
GM2_2s3s done
GM2_3t1s done
GM2_3t2a done
GM2_3t2s done
GM2_3t3t done
GM2_3t3a done
GM2_3t3s done
GM2_3a1s done
GM2_3a2a done
GM2_3a2s done
GM2_3a3t done
GM2_3a3a done
GM2_3a3s done
GM2_3s1s done
GM2_3s2a done
GM2_3s2s done
GM2_3s3t done
GM2_3s3a done
GM2_3s3s done


### Create double-layer matrices

In [20]:
for mode1 in modes:
    for mode2 in modesV:
        if mode2 == '2s':
            globals()['matKM2_'+mode1+mode2] = 8*pi*eta*b/3 * globals()['matGM2_'+mode1+mode2]
            print('KM2_'+mode1+mode2+' done')
            
        elif mode2 == '3t':
            globals()['matKM2_'+mode1+mode2] = 6*pi*eta*b/5 * globals()['matGM2_'+mode1+mode2]
            print('KM2_'+mode1+mode2+' done')

KM2_1s2s done
KM2_1s3t done
KM2_2a2s done
KM2_2a3t done
KM2_2s2s done
KM2_2s3t done
KM2_3t2s done
KM2_3t3t done
KM2_3a2s done
KM2_3a3t done
KM2_3s2s done
KM2_3s3t done


### Delta tensors

In [21]:
## Delta tensors as 'identity' matrices
def Delta1(alpha, beta):
    return KroneckerDelta[alpha,beta]

def Delta2(alpha, gamma1, beta, kappa1):
    return (KroneckerDelta[alpha,beta]*KroneckerDelta[gamma1,kappa1]/2
            + KroneckerDelta[alpha,kappa1]*KroneckerDelta[gamma1,beta]/2
            -KroneckerDelta[alpha,gamma1]*KroneckerDelta[beta,kappa1]/3)

def Delta3(alpha, gamma1, gamma2, beta, kappa1, kappa2):
    return (KroneckerDelta[alpha,beta]*KroneckerDelta[gamma1,kappa1]*KroneckerDelta[gamma2,kappa2] 
                + KroneckerDelta[alpha,beta]*KroneckerDelta[gamma1,kappa2]*KroneckerDelta[gamma2,kappa1] 
                + KroneckerDelta[alpha,kappa1]*KroneckerDelta[gamma1,beta]*KroneckerDelta[gamma2,kappa2] 
                + KroneckerDelta[alpha,kappa1]*KroneckerDelta[gamma1,kappa2]*KroneckerDelta[gamma2,beta] 
                + KroneckerDelta[alpha,kappa2]*KroneckerDelta[gamma1,beta]*KroneckerDelta[gamma2,kappa1]
                + KroneckerDelta[alpha,kappa2]*KroneckerDelta[gamma1,kappa1]*KroneckerDelta[gamma2,beta])/6 - (
        (KroneckerDelta[beta,kappa1]*KroneckerDelta[gamma2,kappa2] + KroneckerDelta[beta,kappa2]*KroneckerDelta[gamma2,kappa1]
         + KroneckerDelta[kappa1,kappa2]*KroneckerDelta[gamma2,beta])*KroneckerDelta[alpha,gamma1]
        +(KroneckerDelta[beta,kappa1]*KroneckerDelta[alpha,kappa2] + KroneckerDelta[beta,kappa2]*KroneckerDelta[alpha,kappa1]
          + KroneckerDelta[kappa1,kappa2]*KroneckerDelta[alpha,beta])*KroneckerDelta[gamma1,gamma2]
        +(KroneckerDelta[beta,kappa1]*KroneckerDelta[gamma1,kappa2] + KroneckerDelta[beta,kappa2]*KroneckerDelta[gamma1,kappa1]
          + KroneckerDelta[kappa1,kappa2]*KroneckerDelta[gamma1,beta])*KroneckerDelta[alpha,gamma2])/15

In [22]:
n_jobs = 1

with parallel_backend('multiprocessing'):
    listDelta1 = Parallel(n_jobs=n_jobs)(delayed(Delta1)(ki,kj) for ki in range(3)
                                                                     for kj in range(3))
matDelta1   = sympy.Matrix(listDelta1).reshape(3,3)

##
with parallel_backend('multiprocessing'):
    listDelta2 = Parallel(n_jobs=n_jobs)(delayed(Delta2)(ki,li, kj,lj) for ki in range(2)
                                                                            for li in range(ki,3)
                                                                                for kj in range(2)
                                                                                  for lj in range(kj,3))
matDelta2   = sympy.Matrix(listDelta2).reshape(5,5)

##
with parallel_backend('multiprocessing'):
    listDelta3 = Parallel(n_jobs=n_jobs)(delayed(Delta3)(ki,li,mi, kj,lj,mj) for ki in range(2)
                                                                                for li in range(ki,2)
                                                                                    for mi in range(li,3)
                                                                                        for kj in range(2)
                                                                                            for lj in range(kj,2)
                                                                                                for mj in range(lj,3))
matDelta3   = sympy.Matrix(listDelta3).reshape(7,7)

### Vectors

In [23]:
## In python code will be force[i,0], so have to create force as np.zeros([3*Np,1]) object instead of np.zeros([3*Np])
force  = sympy.Matrix(sympy.MatrixSymbol('force',3,1)) ##create an actual Matrix from it, have to pass to 'Matrix' 
torque = sympy.Matrix(sympy.MatrixSymbol('torque',3,1))
VH     = sympy.Matrix(sympy.BlockDiagMatrix(matDelta2.inv(), matDelta1.inv(), matDelta2.inv(), matDelta3.inv()))*sympy.Matrix(sympy.MatrixSymbol('VH',20,1))
FH     = sympy.Matrix(sympy.BlockDiagMatrix(matDelta2.inv(), matDelta1.inv(), matDelta2.inv(), matDelta3.inv()))*sympy.Matrix(sympy.MatrixSymbol('FH',20,1))

### Create block matrices and multiply them with vectors

In [24]:
matGM2_1sH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(matGM2_1s2s), sympy.Matrix(matGM2_1s3t), sympy.Matrix(matGM2_1s3a), sympy.Matrix(matGM2_1s3s)]))
matGM2_2aH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(matGM2_2a2s), sympy.Matrix(matGM2_2a3t), sympy.Matrix(matGM2_2a3a), sympy.Matrix(matGM2_2a3s)]))

matKM2_1sH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(matKM2_1s2s), sympy.Matrix(matKM2_1s3t), sympy.Matrix(sympy.ZeroMatrix(3,12))]))
matKM2_2aH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(matKM2_2a2s), sympy.Matrix(matKM2_2a3t), sympy.Matrix(sympy.ZeroMatrix(3,12))]))

matGM2_H1s = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matGM2_2s1s)],
                              [sympy.Matrix(matGM2_3t1s)],
                              [sympy.Matrix(matGM2_3a1s)],
                              [sympy.Matrix(matGM2_3s1s)]]))
matGM2_H2a = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matGM2_2s2a)],
                              [sympy.Matrix(matGM2_3t2a)],
                              [sympy.Matrix(matGM2_3a2a)],
                              [sympy.Matrix(matGM2_3s2a)]]))

matGM2_HH = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matGM2_2s2s), sympy.Matrix(matGM2_2s3t), sympy.Matrix(matGM2_2s3a), sympy.Matrix(matGM2_2s3s)],
                             [sympy.Matrix(matGM2_3t2s), sympy.Matrix(matGM2_3t3t), sympy.Matrix(matGM2_3t3a), sympy.Matrix(matGM2_3t3s)],
                             [sympy.Matrix(matGM2_3a2s), sympy.Matrix(matGM2_3a3t), sympy.Matrix(matGM2_3a3a), sympy.Matrix(matGM2_3a3s)],
                             [sympy.Matrix(matGM2_3s2s), sympy.Matrix(matGM2_3s3t), sympy.Matrix(matGM2_3s3a), sympy.Matrix(matGM2_3s3s)]]))

matKM2_HH_nonzero = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matKM2_2s2s), sympy.Matrix(matKM2_2s3t)],
                                     [sympy.Matrix(matKM2_3t2s), sympy.Matrix(matKM2_3t3t)],
                                     [sympy.Matrix(matKM2_3a2s), sympy.Matrix(matKM2_3a3t)],
                                     [sympy.Matrix(matKM2_3s2s), sympy.Matrix(matKM2_3s3t)]]))
matKM2_HH = sympy.Matrix(sympy.BlockMatrix([matKM2_HH_nonzero, sympy.Matrix(sympy.ZeroMatrix(20,12))]))

# self-interaction

g2s = 3/(20*pi*eta*b)
g3t = 1/(2*pi*eta*b)
g3a = 3/(2*pi*eta*b)
g3s = 6/(7*pi*eta*b)

matGM2_oHH = sympy.Matrix(sympy.BlockDiagMatrix(g2s*matDelta2,
                                             g3t*matDelta1,
                                             g3a*matDelta2,
                                             g3s*matDelta3))


halfMinusk2s = S(3)/5 #0.6
halfMinusk3t = S(2)/5 #0.4
halfMinusk3a = S(4)/5 #0.8
halfMinusk3s = S(19)/35

matKM2_oHH = sympy.Matrix(sympy.BlockDiagMatrix(halfMinusk2s*matDelta2,
                                             halfMinusk3t*matDelta1,
                                             halfMinusk3a*matDelta2,
                                             halfMinusk3s*matDelta3))

In [83]:
## a) can ignore ls symmetries, multiply by (full) Delta tensors and then pick unique elements
## b) return matrices as sympy.Matrix, instead of Matrix. 

In [26]:
vecGM2_H1sF = matGM2_H1s*force
vecGM2_H2aT = matGM2_H2a*torque

vecKM2_HHVH = matKM2_HH*VH

vecGM2_HHFH = matGM2_HH*FH

vecGM2_1s1sF = sympy.Matrix(matGM2_1s1s)*force
vecGM2_2a1sF = sympy.Matrix(matGM2_2a1s)*force

vecGM2_1s2aT = sympy.Matrix(matGM2_1s2a)*torque
vecGM2_2a2aT = sympy.Matrix(matGM2_2a2a)*torque

vecGM2_1sHFH = matGM2_1sH*FH
vecGM2_2aHFH = matGM2_2aH*FH

vecKM2_1sHVH = matKM2_1sH*VH
vecKM2_2aHVH = matKM2_2aH*VH

In [28]:
matGM2_1s1s

[(1/8)*(8*xi + (-160/9)*b**2*xi**3)/(pi**(3/2)*eta), 0, 0]
[0, (1/8)*(8*xi + (-160/9)*b**2*xi**3)/(pi**(3/2)*eta), 0]
[0, 0, (1/8)*(8*xi + (-160/9)*b**2*xi**3)/(pi**(3/2)*eta)]

_____________

### Write symbolic results to file as reusable python functions

In [44]:
ME_file = "M2r0.py"
dimH = 20

with open(ME_file, "w") as text_file:
    print("import numpy as np", file=text_file)
    print("from math import *", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def GM2_1s1sF(v, xi, b,eta, force):", file=text_file)
    print("    v -= np.array({})".format(np.reshape(np.asarray(vecGM2_1s1sF),(3,)).tolist()), file=text_file) ##cannot mulitply list by 0.5/b 
    print("    return\n", file=text_file)
    
    print("def GM2_2a1sF(o, xi, b,eta, force):", file=text_file)
    print("    o -= 0.5/b*np.array({})".format(np.reshape(np.asarray(vecGM2_2a1sF),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GM2_1s2aT(v, xi, b,eta, torque):", file=text_file)
    print("    v -= 1./b*np.array({})".format(np.reshape(np.asarray(vecGM2_1s2aT),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GM2_2a2aT(o, xi, b,eta, torque):", file=text_file)
    print("    o -= 0.5/(b*b)*np.array({})".format(np.reshape(np.asarray(vecGM2_2a2aT),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GM2_H1sF(gh1sf, xi, b,eta, force):", file=text_file)
    print("    gh1sf[:] -= {}".format(np.reshape(np.asarray(vecGM2_H1sF),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GM2_H2aT(gh2at, xi, b,eta, torque):", file=text_file)
    print("    gh2at[:] -= {}".format(np.reshape(np.asarray(vecGM2_H2aT),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def KM2_HHVH(khhvh, xi, b,eta, VH):", file=text_file)
    print("    khhvh[:] -= {}".format(np.reshape(np.asarray(vecKM2_HHVH),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GM2_HHFH(ghhfh, xi, b,eta, FH):", file=text_file)
    print("    ghhfh[:] -= {}".format(np.reshape(np.asarray(vecGM2_HHFH),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GM2_1sHFH(v, xi, b,eta, FH):", file=text_file)
    print("    v += np.array({})".format(np.reshape(np.asarray(vecGM2_1sHFH),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GM2_2aHFH(o, xi, b,eta, FH):", file=text_file)
    print("    o += 0.5/b*np.array({})".format(np.reshape(np.asarray(vecGM2_2aHFH),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def KM2_1sHVH(v, xi, b,eta, VH):", file=text_file)
    print("    v -= np.array({})".format(np.reshape(np.asarray(vecKM2_1sHVH),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def KM2_2aHVH(o, xi, b,eta, VH):", file=text_file)
    print("    o -= 0.5/b*np.array({})".format(np.reshape(np.asarray(vecKM2_2aHVH),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
#filedata = filedata.replace('sqrt', 'numpy.sqrt')

filedata = re.sub("force\[(\d), 0\]", r"force[\1]", filedata)
filedata = re.sub("torque\[(\d), 0\]", r"torque[\1]", filedata)
filedata = re.sub("VH\[(\d||\d\d), 0\]", r"VH[\1]", filedata)
filedata = re.sub("FH\[(\d||\d\d), 0\]", r"FH[\1]", filedata)

with open(ME_file, 'w') as file:
    file.write(filedata)

______________________________________
_______________________

In [36]:
simplifiedM2_3s1s

[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]

In [43]:
sympy.limit(matM2_3s1s[2,2].subs({xij:r0, yij:r0, zij:r0}), r0, 0)

0